# **HW2: Decision Tree** 
In *assignment 2*, you need to:

1.   Implement the decision tree in 3 steps (with the example data)


*   Step 1: calculate the entropy
*   Step 2: search for the best split
*   Step 3: build the decision tree

2.   Predict the patients' death (hospDIED) in the *MIMIC* dataset

Please fill in your **studentID** here.

In [218]:
STUDENT_ID = '108062138'
print(STUDENT_ID)

108062138


# **1. Implement the Decision Tree**
In the first part, you need to implement the decision tree by completing the given funcitions.

Also, you need to run those functions with given input variables and save the output to the implementation csv file **[STUDENT_ID]_implementation.csv**.

Implement a binary decision tree to classify *Restaurant* dataset.

## Import Packages

Note: You **cannot** import any other packages in the first part (implementation)!

In [219]:
import numpy as np
import pandas as pd
import random

## Load the Example Data
First, load the *Restaurant waiting* dataset: **data.csv**

In [220]:
example = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/data.csv')
example

,Unnamed: 0,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,X1,T,F,F,T,Some,High,F,T,French,8,T
1,X2,T,F,F,T,Full,Low,F,F,Thai,40,F
2,X3,F,T,F,F,Some,Low,F,F,Burger,8,T
3,X4,T,F,T,T,Full,Low,F,F,Thai,12,T
4,X5,T,F,T,F,Full,High,F,T,French,70,F
5,X6,F,T,F,T,Some,Medium,T,T,Italian,3,T
6,X7,F,T,F,F,None,Low,T,F,Burger,7,F
7,X8,F,F,F,T,Some,Medium,T,T,Thai,6,T
8,X9,F,T,T,F,Full,Low,T,F,Burger,80,F
9,X10,T,T,T,T,Full,High,F,T,Italian,20,F


In [221]:
example = example.drop(['Unnamed: 0'], axis=1)

In [222]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

example['Alternate'] = labelencoder.fit_transform(example['Alternate'])
example['Bar'] = labelencoder.fit_transform(example['Bar'])
example['Friday'] = labelencoder.fit_transform(example['Friday'])
example['Hungry'] = labelencoder.fit_transform(example['Hungry'])
example['Patrons'] = labelencoder.fit_transform(example['Patrons'])
example['Price'] = labelencoder.fit_transform(example['Price'])
example['Raining'] = labelencoder.fit_transform(example['Raining'])
example['Reservation'] = labelencoder.fit_transform(example['Reservation'])
example['Type'] = labelencoder.fit_transform(example['Type'])

example

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
2,0,1,0,0,2,1,0,0,0,8,T
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
5,0,1,0,1,2,2,1,1,2,3,T
6,0,1,0,0,1,1,1,0,0,7,F
7,0,0,0,1,2,2,1,1,3,6,T
8,0,1,1,0,0,1,1,0,0,80,F
9,1,1,1,1,0,0,0,1,2,20,F


## Calculating the Entropy *(20%)*


In [223]:
# split the data by given attribute and its threshold
def partition(data, column, threshold):
    """
      The *partition* function will split the input data into 2 branches.
        args:
        *   data(DataFrame): the input data
        *   column(str): the attribute(column name)
        *   threshold(float): the column's threshold for splitting the data
        returns:
        *   match_branch(DataFrame): the divided data that matches the assigned column's threshold
        *   false_branch(DataFrame): the divided data that doesn't match the assigned column's threshold
    """
    matchBranch = data[data[column]<=threshold]
    falseBranch = data[data[column]>threshold]
    return matchBranch, falseBranch

If we try *partition(example, 'Patrons', 1.5)*:

split example to 

match_left(Patrons<= 1.5) (Patrons != Some) and  
false_right(Patrons > 1.5) (Patrons == Some)

column: Patrons, threshold: 1.5



In [224]:
match_left, false_right = partition(example, 'Patrons', 1.5)

In [225]:
def entropy(data):
    """
      The *entropy* function will calculate the entropy of the node(data)
      args:
      *    data(DataFrame): the data you're calculating for the entropy
      output:
      *    entropy(float): the node(data)'s entropy     
    """
    if data.shape[0]==0:
        return 0
    numCorrect = 0
    numNotCorrect = 0
    for index, row in data.iterrows():
        if row['Wait']=='T':
            numCorrect=numCorrect+1
        else:
            numNotCorrect=numNotCorrect+1
    total = numCorrect+numNotCorrect
    p = numCorrect / total
    if p==1 or p==0:
        return 0
    entropy = -p*np.log2(p)-(1-p)*np.log2(1-p)
    return entropy

An example of the output from entropy (Use this to check if your output is correct): 

In [226]:
print(example.shape)
match, false = partition(example, 'Hungry', 0.5)
entropy1 = entropy(match)
print(entropy1)

(12, 11)
0.7219280948873623


In [227]:
#implementation 1: calculate the entropy of 'false_right' data
ans_entropy = entropy(false)

## Find the Best Split *(20%)*

In [228]:
# search for the best attribute and the value(threshold) to split the data
def findBestSplit(data):
    """
    The *findBestSplit* function finds the best combination of attribute and value(with the largest reduction in entropy) to split the data.
    args:
    *   data(DataFrame): the data you try to split(build the decision tree)
    output:
    *   column_best(str): the attribute(column) split with the largest reduction in entropy
    *   value_best(float): the value(threshold) of the column_best attribute to split the data
    """
    all_entropy = 9999        #the overall entropy : save the value of the smallest entropy
    column_best = 0
    value_best = 0
    cols = data.columns
    
    for col in cols:
        if col == 'Wait':
            break;
            
        st = []
        for index, row in data.iterrows():
            st.append(row[col])
        
        st = sorted(st)
        mls = list(st)
        mls = np.unique(mls)
        n = len(mls)
        
        Thresholds = []
        
        for i in range(1,n):
            md = (mls[i]+mls[i-1])/2
            Thresholds.append(float(md))
        
        for threshold in Thresholds:
            match, false = partition(data,col,threshold)
            entropyP = entropy(match)
            entropyN = entropy(false)
            p = match.shape[0]
            n = false.shape[0]
            curEntropy = p/(n+p) * entropyP + n/(n+p) * entropyN   
            if all_entropy > curEntropy:
                all_entropy = curEntropy
                column_best = col
                value_best = threshold
    return column_best, value_best

An example of the output from findBestSplit (Use this to check if your output is correct):

In [230]:
print(example)
column_best, value_best = findBestSplit(example)
print('for col: ',column_best)
print('for value: ',value_best)

    Alternate  Bar  Friday  Hungry  Patrons  Price  Raining  Reservation  \
0           1    0       0       1        2      0        0            1   
1           1    0       0       1        0      1        0            0   
2           0    1       0       0        2      1        0            0   
3           1    0       1       1        0      1        0            0   
4           1    0       1       0        0      0        0            1   
5           0    1       0       1        2      2        1            1   
6           0    1       0       0        1      1        1            0   
7           0    0       0       1        2      2        1            1   
8           0    1       1       0        0      1        1            0   
9           1    1       1       1        0      0        0            1   
10          0    0       0       0        1      1        0            0   
11          1    1       1       1        0      1        0            0   

    Type  W

In [231]:
#implementation 2: Find the best split of the 'match' data
ans_column, ans_value = findBestSplit(false)
print('for col: ',ans_column)
print('for value: ',ans_value)

for col:  WaitEstimate
for value:  16.0


## Decision Tree Building *(30%)*

Use the above functions to help building the decision tree





In [232]:
def buildTree(data, depth):
    """
      The *buildTree* function builds the decision tree
      args:
      *     df(DataFrame): the data you want to apply the decision tree
      *     depth(int) : the depth of your tree
      output:
      *     decisionSubTree(dict): the decision tree structure including root, branch, leaf(with the attributes and thresholds)
      *     features(list): the features(attributes) name in the decision tree structure(from root to branch and leaf)
        *     thresholds(list): the corresponding thresholds for the features in the 'features' list
    """
    print(data.shape[0])
    
    column_best, value_best = findBestSplit(data)
    features = []
    thresholds = []
    features.append(column_best)
    thresholds.append(value_best)
    
    numT=0
    numN=0
    for index, row in data.iterrows():
        if row['Wait']=='T':
            numT=numT+1
        else:
            numN=numN+1
    if numT == data.shape[0]:
        return 'T',[],[]
    if numN == data.shape[0]:
        return 'F',[],[]
    if depth==1:
        if numT>numN:
            return 'T',[],[]
        else: return 'F',[],[]
    
    
    match_left, false_right = partition(data, column_best, value_best)
    
    lTree, lFeatures, lThresholds = buildTree(match_left ,depth-1)
    rTree, rFeatures, rThresholds = buildTree(false_right,depth-1)
    #features.append(lFeatures)
    #features.append(rFeatures)
    #thresholds.append(lThresholds)
    #thresholds.append(rThresholds)
    features = features + lFeatures + rFeatures
    thresholds = thresholds + lThresholds + rThresholds
    decisionSubTree = [column_best,value_best,lTree,rTree]
    return decisionSubTree, features, thresholds

An example of the output from buildTree (Use this to check if your output is correct): 

In [233]:
tree, features, thresholds= buildTree(example, 2)
print(tree)
print(features)
print(thresholds)

12
8
4
['Patrons', 1.5, 'F', 'T']
['Patrons']
[1.5]


In [234]:
#implementation 3: decision tree building (depth=10)
ans_tree, ans_features, ans_thresholds = buildTree(example,10)

print(ans_tree)
print(ans_features)
print(ans_thresholds)

12
8
4
4
1
3
1
2
4
['Patrons', 1.5, ['Hungry', 0.5, 'F', ['Friday', 0.5, 'F', ['Price', 0.5, 'F', 'T']]], 'T']
['Patrons', 'Hungry', 'Friday', 'Price']
[1.5, 0.5, 0.5, 0.5]


In [235]:
#save implementation csv
ans_path = STUDENT_ID + '_implementation.csv'

imp = []
imp.append(ans_entropy)
imp.append(ans_column)
imp.append(ans_value)
for i in range(len(ans_features)):
  imp.append(ans_features[i])
for m in range(len(ans_thresholds)):
  imp.append(ans_thresholds[m])
print(imp)
pd.DataFrame(imp).to_csv(ans_path, header = None, index = None)

[0.863120568566631, 'WaitEstimate', 16.0, 'Patrons', 'Hungry', 'Friday', 'Price', 1.5, 0.5, 0.5, 0.5]


# **2. Classification with the MIMIC Dataset**
In the second part, you need to classify **'hospDIED'(death)** in the MIMIC dataset.

Please put the classification result in a csv file. (**[STUDENTID]_prediction.csv**)

**Note:** Decision tree is recommended but not mandatory.

## Prediction (Performance) *(20%)*

The **y_test** (hospDIED) of this data is hidden, you need to use the x_test.csv to predict the y_test.

**Note:** You **can** now import the packages you need here!

Get the MIMIC data here!

Data Description: 

[Data Description](https://docs.google.com/spreadsheets/d/1pxqxQFhIcv_hrgWEtwhXE6zBVQ5ISa-13PIhvXMtWCY/edit?usp=sharing) (You can find the data description here.)

**Note:**
*   You can select the features you want to use.
*   You can use any ML models to predict the y_test.

  (However, there is a 10% bonus if you **visualize** the decision tree in this part)


[ref]: *Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.*



In [237]:
# inline plotting instead of popping out
%matplotlib inline

import os
from IPython.display import display

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.datasets import load_wine, load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz

Complete your model with validation:

In [247]:
#Read data
X = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_train.csv')
y = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/y_train.csv')
X = X.drop('indextime', 1)
display(X.head())
display(y.head())
#Split the data into training and validation sets
#Build you model and evaluate it on your validation set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

print('#Training data points: %d' % X_train.shape[0])
print('#Testing data points: %d' % X_test.shape[0])
print('Class labels:', np.unique(y))

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=3, 
                              random_state=0)
tree.fit(X_train, y_train)


,subject_id,age,los,CA,DNR,CMO,DNI,female,first_careunit,insurance,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10246985,48,1.826250,0,0,0,0,1,10,1,...,2,9,9,9,9,0,9,9,0,9
1,14538806,59,10.581123,0,0,0,0,1,10,1,...,4,4,4,4,4,4,4,4,4,4
2,12850130,73,2.065255,0,0,0,0,1,4,1,...,4,4,4,4,4,4,4,4,4,4
3,11810761,84,1.435428,0,0,0,0,1,10,3,...,0,9,9,0,9,0,9,1,9,9
4,13109236,66,1.443414,0,0,0,0,0,1,1,...,4,4,4,4,4,4,4,4,4,4


,hospDIED
0,0
1,0
2,0
3,0
4,0


#Training data points: 17248
#Testing data points: 7393
Class labels: [0 1]


DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

Make the final prediction from your model:

In [260]:
print(tree.score(X_test,y_test))

y_pred = tree.predict(X_test)
y_pred = y_pred.reshape(y_pred.shape[0],1)
print(y_pred.shape)
print(y_test.shape)
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Accuracy (tree): %.2f' % ((y_test == y_pred).sum() / y_test.shape[0]))

# a more convenient way to evaluate a trained model is to use the sklearn.metrics 
from sklearn.metrics import accuracy_score
print('Accuracy (tree, sklearn): %.2f' % accuracy_score(y_test, y_pred))

from sklearn.metrics import f1_score
print('f1 score: ',f1_score(y_test,y_pred))

0.8806979575273908
(7393, 1)
(7393, 1)
Misclassified samples: 882
Accuracy (tree): 0.88
Accuracy (tree, sklearn): 0.88
f1 score:  0.43461538461538457


In [258]:
#final prediction with your decision tree(optional)
def treePredictions(df, tree):
    predictions = tree.predict(df)
    return predictions

x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')
x_test = x_test.drop('indextime', 1)
y_pred = treePredictions(x_test, tree)

To export your predcition as a CSV file and hand in the CSV on elearn

In [259]:
output_path = STUDENT_ID + '_prediction.csv'

tree_test_pred = pd.DataFrame( {'subject_id': x_test.subject_id,
                  'prediction': y_pred } )
tree_test_pred.to_csv(output_path, index = False)

## Visualizing the Decision Tree *(10% bonus)*

**Note:** Save the visualization result image as **[STUDENT_ID]_visualization.png**

*   Your visualization image of the decision tree can contain **five** layers at most.


In [261]:
#Decision Tree Visualization
if not os.path.exists("./output/") : os.mkdir("./output/")
export_graphviz(
    tree,
    out_file='./output/tree.dot', 
    feature_names=X.columns.values
)

# Report *(10%)*

Report should be submitted as a pdf file! (**[STUDENT_ID]_report.pdf**)

*   List the top 3 splitting features and their thresholds of your model
*   Briefly describe how you build the decision tree
*   Describe if you apply any improvement on your decision tree model
*   If you preprocess the MIMIC data in the second part (selecting features...), describe the work and reasons
*   Summarize your work
*   Do not exceed 2 pages!






# Save the Code File
Please save your code and submit it as an ipynb file! (**[STUDENT_ID]_hw2.ipynb**)